This is my first notebook for pyiron-continuum project

Now let's try to import te pyiron-continuum library

In [1]:
from pyiron_continuum import Project
from damask import Rotation

In [2]:
# let's create the project and related folder
pr = Project("damask_job")

In [3]:
# let's clean the previous content
pr.remove_jobs_silently(recursive=True)

In [4]:
# now we can create a DAMASK type job
job = pr.create.job.DAMASK("damask_job")

In [5]:
# now we can define our geometry
grid = pr.continuum.damask.Grid.via_voronoi_tessellation(box_size=1.0e-5,grid_dim=16,num_grains=4)
# assing the grid to our job
job.grid = grid

job.grid.save("mygeo") # save your geometry to vti file

In [6]:
# now we can define the homogenization method
homogenization = pr.create.DAMASK.homogenization(method='SX',parameters={'N_constituents':1,
                                                                         "mechanical":{"type":"pass"}})

In [7]:
# is this duplicated?
homogenization = pr.continuum.damask.Homogenization(method='SX',parameters={'N_constituents':1,
                                                                         "mechanical":{"type":"pass"}})

In [8]:
# now we can define the elasticity
elasticity = pr.continuum.damask.Elasticity(type='Hooke',C_11= 106.75e9,C_12= 60.41e9, C_44=28.34e9) # unit is Pa

In [9]:
# now we can define the plasticity model and input the related parameters
plasticity = pr.continuum.damask.Plasticity(N_sl=[12], a_sl=2.25, 
                                   atol_xi=1.0, dot_gamma_0_sl=0.001,
                                   h_0_sl_sl=75e6, h_sl_sl=[1, 1, 1.4, 1.4, 1.4, 1.4],
                                   n_sl=20, output=['xi_sl'], type='phenopowerlaw',
                                   xi_0_sl=[31e6], xi_inf_sl=[63e6])

In [10]:
# now we can define the phase for current job
phase = pr.continuum.damask.Phase(composition='Aluminum', lattice= 'cF',
                         output_list=['F', 'P', 'F_e', 'F_p', 'L_p', 'O'],
                         elasticity=elasticity, plasticity=plasticity)

In [11]:
# create the orientation for grains in our RVE
rotation = pr.continuum.damask.Rotation(Rotation.from_random,10)
# then we can setup the material yaml

# this one seems only give one grain?
material = pr.continuum.damask.Material(rotation=rotation,elements=['Aluminum'],homogenization=homogenization,phase=phase)
print(material)

material = pr.continuum.damask.Material([rotation],elements=['Aluminum'],homogenization=homogenization,phase=phase)

material:
  - constituents:
      - O: [0.29485788241683925, -0.11002293577315385, -0.8968088054809722, 0.31094653751457924]
        phase: Aluminum
        v: 1.0
    homogenization: SX

phase:
  Aluminum:
    lattice: cF
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 106750000000.0, C_12: 60410000000.0, C_44: 28340000000.0}
      plasticity:
        N_sl: [12]
        a_sl: 2.25
        atol_xi: 1.0
        dot_gamma_0_sl: 0.001
        h_0_sl_sl: 75000000.0
        h_sl_sl: [1, 1, 1.4, 1.4, 1.4, 1.4]
        n_sl: 20
        output: [xi_sl]
        type: phenopowerlaw
        xi_0_sl: [31000000.0]
        xi_inf_sl: [63000000.0]

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}



In [12]:
# now we can assign the material to our job
job.material = material
job.material.save() # this will save the current configuration to 'material.yaml'

In [13]:
# you can check the material via:
print(job.material)

material:
  - constituents:
      - O: [0.29485788241683925, -0.11002293577315385, -0.8968088054809722, 0.31094653751457924]
        phase: Aluminum
        v: 1.0
    homogenization: SX
  - constituents:
      - O: [0.6764648880468285, 0.005846125599514117, 0.5930185897320445, 0.436680696032563]
        phase: Aluminum
        v: 1.0
    homogenization: SX
  - constituents:
      - O: [0.6091808046230107, -0.098962159404594, 0.04210506632706635, -0.7857050347773229]
        phase: Aluminum
        v: 1.0
    homogenization: SX
  - constituents:
      - O: [0.03585816784591728, -0.7626476538598055, -0.5578521402612656, 0.32539781417027586]
        phase: Aluminum
        v: 1.0
    homogenization: SX
  - constituents:
      - O: [0.9558095073329061, -0.2479119070960596, 0.07969224848609123, -0.1364441920466124]
        phase: Aluminum
        v: 1.0
    homogenization: SX
  - constituents:
      - O: [0.21213191076892593, -0.5040586309085399, 0.5905879041387748, -0.5934061648863299]
  

In [14]:
# then, we can create the loading step
myloading = [{'mech_bc_dict':{'dot_F':[1e-3,0,0, 0,'x',0,  0,0,'x'],
                            'P':['x','x','x', 'x',0,'x',  'x','x',0]},
            'discretization':{'t': 10.,'N': 40},
            'additional': {'f_out': 4}
           },{'mech_bc_dict':{'dot_F':[1e-3,0,0, 0,'x',0,  0,0,'x'],
                              'P':['x','x','x', 'x',0,'x',  'x','x',0]},
            'discretization':{'t': 60.,'N': 60},
            'additional': {'f_out': 4}
           }]
# choose a solver(currently, only mechanics)
print(job.list_solvers()) # list out all the solver

[{'mechanical': 'spectral_basic'}, {'mechanical': 'spectral_polarization'}, {'mechanical': 'FEM'}]


In [15]:
mechsolver = job.list_solvers()[0] # here we choose the mechanics solver

# assign the loaidng to our job
job.loading = pr.continuum.damask.Loading(solver=mechsolver, load_steps=myloading)

job.loading.save("myload.yaml") # save our loading yaml file

In [20]:
job.run()